In [ ]:
import cv2
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from skimage.segmentation import felzenszwalb

# Load pre-trained CNN (VGG16)
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

# Load classifier (SVM or Softmax)
classifier = SVC()

# Load training data (images and corresponding labels)
train_images = ['train_img1.jpg', 'train_img2.jpg', ...]  # List of training image paths
train_labels = ['label1', 'label2', ...]  # List of corresponding labels

# Extract features and labels from training data
features_train = []
labels_train = []
img= cv2.imread(r"D:\Datasets\Aquarium Combined.v2-raw-1024.tensorflow\train\IMG_2284_jpeg_jpg.rf.99de11cb5727748bd3eae3afe7b415e6.jpg")
for img_path, label in zip(img, 1):
    image = cv2.imread(img_path)
    
    regions = felzenszwalb(image, scale=100, sigma=0.9, min_size=50)
    
    for region in regions:
        (x, y, w, h) = cv2.boundingRect(region)
        
        roi = image[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi, (224, 224))  # Resize region to fit VGG input size
        roi_preprocessed = np.expand_dims(cv2.cvtColor(roi_resized, cv2.COLOR_BGR2RGB), axis=0)
        
        features = model.predict(roi_preprocessed)
        features_flattened = features.flatten()
        features_train.append(features_flattened)
        
        labels_train.append(label)

# Train the classifier
classifier.fit(features_train, labels_train)



In [3]:
# Load testing data (images)
img= cv2.imread(r"D:\Datasets\Aquarium Combined.v2-raw-1024.tensorflow\train\IMG_2284_jpeg_jpg.rf.99de11cb5727748bd3eae3afe7b415e6.jpg")

# Perform object detection and classification on testing data

regions = felzenszwalb(img, scale=100, sigma=0.9, min_size=50)
print(regions)
for region in regions:
    (x, y, w, h) = cv2.boundingRect(region)
    roi = image[y:y+h, x:x+w]
    roi_resized = cv2.resize(roi, (224, 224))  # Resize region to fit VGG input size
    roi_preprocessed = np.expand_dims(cv2.cvtColor(roi_resized, cv2.COLOR_BGR2RGB), axis=0)
    features = model.predict(roi_preprocessed)
    features_flattened = features.flatten()
    prediction = classifier.predict([features_flattened])[0]

    # Draw bounding box and label on the image
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, f'Class: {prediction}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image with bounding boxes and labels
cv2.imshow('Object Detection', image)
cv2.waitKey(0)

cv2.destroyAllWindows()

[[   0    0    0 ...   37   37   37]
 [   0    0    0 ...   37   37   37]
 [   0    0    0 ...   37   37   37]
 ...
 [1364 1364 1364 ... 1353 1353 1353]
 [1364 1364 1364 ... 1353 1353 1353]
 [1364 1364 1364 ... 1353 1353 1353]]


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:874: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::pointSetBoundingRect'


In [2]:
import cv2
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from skimage.segmentation import felzenszwalb

# Load pre-trained CNN (VGG16)
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

# Load classifier (SVM or Softmax)
classifier = SVC()

# Load image
image= cv2.imread(r"D:\Datasets\Aquarium Combined.v2-raw-1024.tensorflow\train\IMG_2284_jpeg_jpg.rf.99de11cb5727748bd3eae3afe7b415e6.jpg")


# Perform Selective Search Fast
regions = felzenszwalb(image, scale=100, sigma=0.9, min_size=50)

# Extract features and classify each region proposal
for region in regions:
    (x, y, w, h) = region
    
    # Extract region from the image
    roi = image[y:y+h, x:x+w]
    
    # Resize ROI to fit VGG input size
    roi_resized = cv2.resize(roi, (224, 224))
    
    # Preprocess ROI for VGG input
    roi_preprocessed = np.expand_dims(cv2.cvtColor(roi_resized, cv2.COLOR_BGR2RGB), axis=0)
    roi_preprocessed = preprocess_input(roi_preprocessed)
    
    # Extract features using the pre-trained CNN
    features = model.predict(roi_preprocessed)
    
    # Flatten features
    features_flattened = features.flatten()
    
    # Classify the region using the classifier
    prediction = classifier.predict([features_flattened])[0]
    
    # Draw bounding box and label on the image
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, f'Class: {prediction}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image with bounding boxes and labels
cv2.imshow('Object Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

ValueError: too many values to unpack (expected 4)